In [1]:
import pandas as pd
import numpy as np
from pyscipopt import Model
from pyscipopt import quicksum

# import data

In [2]:
suffix = "_hardcore"

In [3]:
people = pd.read_csv("data_processed/people.csv", index_col="Name")
boats = pd.read_csv('data_processed/boats.csv', index_col='name')
time_prefs = {'comp': pd.read_csv(f'data_processed/comp/time_prefs{suffix}.csv',
                                  header=[0, 1], index_col=0),
              'rec_master': None
              }

## Optim for competitive group

In [4]:
GROUP = 'comp'

In [5]:
prefs = time_prefs[GROUP]
people_set = set(prefs.index)
time_set = set(prefs.columns)
boat_set = set(boats.index)

In [6]:
# number of trainings people ask for (number of first choice)
asked_nb_trainings = prefs.apply(lambda x: x.value_counts()[0], axis=1)

In [7]:
# for each group, dict with key = level of people, values = list of boat skills they can row
boat_class_per_skill = {
    'comp': {
        1: [0, 1],
        2: [2]
    },
    'rec_master': {
        1: [1],
        2: [2],
        3: [2]
    }
}

## create dicts that help generate vars and constraints

In [8]:
# boats_for_person and times_avail_for_person for each person
boats_for_person = {}
times_avail_for_person = {}
for p in people_set:
    skill = people.loc[p]['Level']
    weight = people.loc[p]['Class']

    # mask selecting boats with a skill mathcing the level of the person
    skill_mask = boats['skill'].isin(boat_class_per_skill['comp'][skill])
    # mask selecting boats that match the weight class of the person
    weight_mask = boats[weight] == 1

    matching_boats = boats[skill_mask & weight_mask]
    boats_for_person[p] = set(matching_boats.index)

    # available times for the person
    times_avail_for_person[p] = set(prefs.loc[p][prefs.loc[p].notna()].index)

# people likely to row a given boat at a given time
people_for_boat_time = {}
for b in boat_set:
    skill_mask = people['membership'] == 'comp'
    if boats.loc[b]['skill'] in (0, 1):
        skill_mask &= people['Level'] == 1
    elif boats.loc[b]['skill'] == 2:
        skill_mask &= people['Level'] == 2
    else:
        continue

    weight_mask = pd.Series(data=False, index=people.index)  # neutral mask (does not mask anything)
    for weight in ['L', 'M', 'MH', 'H']:
        if boats.loc[b][weight] == 1:
            weight_mask |= people['Class'] == weight

    people_for_boat = set(people[skill_mask & weight_mask].index)
    # only people who gave time prefs
    people_for_boat &= people_set

    for t in time_set:
        people_for_boat_time[(b, t)] = {p for p in people_for_boat
                                        if not np.isnan(prefs.loc[p, t])}

## Define the model

In [9]:
# definition of the model
model = Model("boat_alloc")

# defining varialbes
variables = {}
for p in people_set:
    for b in boats_for_person[p]:
        for t in times_avail_for_person[p]:
            variables[(p, b, t)] = model.addVar(vtype="B")

# minimal pref score among all people, that we want to maximize
s = model.addVar(vtype="I", name="min_pref", lb=None)


# setting the objective fucntion
total_pref = quicksum(prefs.loc[p, t] * variables[(p, b, t)]
                      for p in people_set
                      for b in boats_for_person[p]
                      for t in times_avail_for_person[p])
model.setObjective(500 * s + total_pref, "maximize")

# adding the constraints
for p in people_set:
    # sum of prefs
    sum_of_prefs = quicksum(prefs.loc[p, t] * variables[(p, b, t)]
                            for b in boats_for_person[p]
                            for t in times_avail_for_person[p])
    # s = min_{p} sum of prefs
    model.addCons(s <= sum_of_prefs)

    # each person trains the number of times they asked
    nb_trainings = quicksum(variables[(p, b, t)]
                            for b in boats_for_person[p]
                            for t in times_avail_for_person[p])
    model.addCons(nb_trainings == asked_nb_trainings[p])

    # each person can only row one boat at a time
    for t in times_avail_for_person[p]:
        nb_boats = quicksum(variables[(p, b, t)]
                            for b in boats_for_person[p])
        model.addCons(nb_boats <= 1)
    
    # not am1 and am2 in the same day
    for day in ['M', 'T', 'W', 'Th', 'F', 'S']:
        if not np.isnan(prefs.loc[p, (day, 'am1')]) and not np.isnan(prefs.loc[p, (day, 'am2')]):
            model.addCons(quicksum(variables[(p, b, (day, 'am1'))] + variables[(p, b, (day, 'am2'))] for b in boats_for_person[p]) <= 1)

# for each boat and time, no more than one person
for b in set(boats[boats['skill'].isin([0, 1, 2])].index):
    for t in time_set:
        nb_people_on_boat_time = quicksum(variables[(p, b, t)]
                                          for p in people_for_boat_time[(b, t)])
        model.addCons(nb_people_on_boat_time <= 1)

## solve the model

In [10]:
# optimize
model.redirectOutput()
model.optimize()

presolving:
(round 1, fast)       0 del vars, 125 del conss, 0 add conss, 1 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 1046 clqs
   (0.0s) running MILP presolver
   (0.1s) MILP presolver (2 rounds): 0 aggregations, 61 fixings, 0 bound changes
(round 2, medium)     61 del vars, 125 del conss, 0 add conss, 2 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 1013 clqs
(round 3, fast)       61 del vars, 158 del conss, 0 add conss, 2 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 1013 clqs
(round 4, exhaustive) 61 del vars, 626 del conss, 0 add conss, 2 chg bounds, 468 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 1013 clqs
   (0.1s) running MILP presolver
   (0.1s) MILP presolver (2 rounds): 0 aggregations, 14 fixings, 0 bound changes
(round 5, medium)     75 del vars, 626 del conss, 0 add conss, 2 chg bounds, 468 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 1011 clqs
(round 6, fast)       75 del vars, 628 del conss, 0 add conss, 2 chg bou

## output solution

In [11]:
status = model.getStatus()
if status == 'optimal':
    # generate resulting CSV file with boat allocation
    levels = [['M', 'T', 'W', 'Th', 'F', 'S'], ['am1', 'am2', 'pm']]
    cols = pd.MultiIndex.from_product(levels, names=['day', 'time'])
    result = pd.DataFrame(columns=cols, index=prefs.index)
    result.index.rename('people', inplace=True)
    
    fairness = pd.DataFrame(0, index=people_set, columns=['nb_asked', 'nb_first', 'nb_second'])
    fairness['nb_asked'] = asked_nb_trainings

    for p in people_set:
        for t in times_avail_for_person[p]:
            boat = {b for b in boats_for_person[p] if model.isEQ(model.getVal(variables[(p, b, t)]), 1)}
            assert len(boat) <= 1
            if boat:
                result.loc[p, t] = boat.pop()
            # if p is rowing at time t
            if model.isEQ(model.getVal(quicksum(variables[(p, b, t)] for b in boats_for_person[p])), 1):
                if prefs.loc[p, t] == 0:
                    fairness.loc[p, 'nb_first'] += 1
                elif prefs.loc[p, t] == -1:
                    fairness.loc[p, 'nb_second'] += 1
                else:
                    raise

    result.to_csv(f'results/comp/results_boat{suffix}.csv')
    result.to_excel(f'results/comp/results_boat{suffix}.xlsx')
    fairness.to_csv(f'results/comp/results_boat{suffix}_fairness.csv')
    fairness.to_excel(f'results/comp/results_boat{suffix}_fairness.xlsx')
    print("Fairness score (0 = optimal) :", model.getVal(s))
    print("total :", model.getVal(total_pref))

else:
    print(f"Solver finished with status {status}")

Fairness score (0 = optimal) : -2.0
total : -24.0


In [12]:
results = pd.concat((result, people), axis=1)
results = results[results['membership'] == 'comp']
results = results.sort_values(['Level', 'Class'])
results = results.drop(columns=['Level', 'Class', 'membership'])
results.columns = pd.MultiIndex.from_tuples(results.columns)

results.to_csv(f'results/comp/results_boat{suffix}_sorted.csv')
results.to_excel(f'results/comp/results_boat{suffix}_sorted.xlsx')

In [13]:
model.getStatus()

'optimal'